In [1]:
import torch
from easy_transformer import EasyTransformer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(f"Using {device} device")
torch.set_grad_enabled(False)

model = EasyTransformer.from_pretrained('gpt2').to(device)

# Convenience function for decoding token
decode = model.tokenizer.decode

# Convenience function for encoding token
def encode(t):
    global model
    result = model.tokenizer.encode(t)
    if len(result) != 1:
        raise Exception(f"Not a single token: {t}")
    return result[0]

unembed = model.unembed.W_U.data
embed = model.embed.W_E.data
d_M = model.cfg.d_model
d_V = model.cfg.d_vocab

print(unembed.shape, d_M, d_V)

/home/giles/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Using cpu device
Loading model: gpt2


Using pad_token, but it is not set yet.


Moving model to device:  cpu
Finished loading pretrained model gpt2 into EasyTransformer!
Moving model to device:  cpu
torch.Size([768, 50257]) 768 50257


In [36]:
# Choose a random set of tokens (TODO: do better than random)
import random
from sklearn.decomposition import SparseCoder

def gen_and_eval_dict(n_dict, n_eval=1000, n_coefs=5):
    random.seed(12345)
    indices = torch.tensor(random.sample(range(d_V), k=n_dict))
    #print([decode(i) for i in indices[:10]])

    dictionary = embed[indices,:]
    #print(dictionary.shape)
    
    eval_ts = torch.tensor(random.sample(range(d_V), k=n_eval))
    vs = embed[eval_ts,:]
    sc = SparseCoder(dictionary=dictionary, transform_n_nonzero_coefs=n_coefs, transform_algorithm='omp')
    transformed = torch.tensor(sc.transform(vs))
    
    #print(transformed.shape, dictionary.shape)
    reconstituted = torch.matmul(transformed, dictionary)
    error = torch.linalg.matrix_norm(reconstituted - vs)
    
    return dictionary, sc, indices, error
    
errors = {}
for n_dict in range(10, 100, 10):
    errors[n_dict] = gen_and_eval_dict(n_dict, n_coefs=5)[3].item()
print(errors)

/home/giles/.local/lib/python3.10/site-packages/sklearn/decomposition/_dict_learning.py:208: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  new_code = orthogonal_mp_gram(
/home/giles/.local/lib/python3.10/site-packages/sklearn/decomposition/_dict_learning.py:208: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  new_code = orthogonal_mp_gram(
/home/giles/.local/lib/python3.10/site-packages/sklearn/decomposition/_dict_learning.py:208: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  new_code = orthogonal_mp_gram(
/home/giles/.local/lib/python3.10/site-packages/sklearn/decomposition/_dict_learning.py:208: RuntimeWarning: Orthogonal matching pursuit ended prematurely du

{10: 112.50882720947266, 20: 110.68318176269531, 30: 109.38329315185547, 40: 108.73686981201172, 50: 108.18726348876953, 60: 107.76912689208984, 70: 107.38787841796875, 80: 107.11817932128906, 90: 106.91816711425781}


/home/giles/.local/lib/python3.10/site-packages/sklearn/decomposition/_dict_learning.py:208: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  new_code = orthogonal_mp_gram(


In [37]:
# Choose another set of tokens
toks = [' peace', ' love', ' war', ' cat', ' dog', ' Mario', ' Giles']
ts = torch.tensor([encode(tok) for tok in toks])
n_t = len(ts)
vs = embed[ts,:]    # gather lookup

dictionary, sc, indices, _ = gen_and_eval_dict(2000)

last_tok = None
transformed = torch.tensor(sc.transform(vs))
for i in range(n_t):
    v = embed[ts[i],:]
    
    print(f'+++{toks[i]}+++')
    nz = torch.nonzero(transformed[i,:])
    values = [(transformed[i,j].item(), decode(indices[j.item()])) for j in nz]
    values.sort(reverse=True)
    for val, tok in values:
        print('    ', f'{tok:20}', val)
    


/home/giles/.local/lib/python3.10/site-packages/sklearn/decomposition/_dict_learning.py:208: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  new_code = orthogonal_mp_gram(


+++ peace+++
      settlers            0.17874906957149506
      conscience          0.16404199600219727
      cessation           0.14536640048027039
     �                    0.11943838745355606
     afety                0.061829060316085815
+++ love+++
      hatred              0.32344797253608704
     -                    0.17184175550937653
      goodness            0.12907937169075012
      Beautiful           0.11889779567718506
     favorite             0.09814704209566116
+++ war+++
      fight               0.29392004013061523
      hatred              0.2270224690437317
      troop               0.16807401180267334
      arms                0.15888924896717072
      Github              -0.12153104692697525
+++ cat+++
      rabbits             0.18691937625408173
      bas                 0.1638754904270172
      sch                 0.16002868115901947
      catch               0.14967380464076996
      fel                 0.14380015432834625
+++ dog+++
      monkey          